<a href="https://colab.research.google.com/github/KavinduPiyumantha/ComposeBottomSheetIssue/blob/master/Open_AI_Fine_Tune_GoogleSheet-to-JSONL_Convert_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Training data - Neluni





In [1]:
# Step 1: Install necessary libraries
!pip install --upgrade gspread pandas gspread-dataframe tiktoken pytz --quiet

# Step 2: Import necessary libraries
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
from google.auth import default
import pandas as pd
import json
from collections import defaultdict
import numpy as np
import tiktoken
from datetime import datetime
import pytz
from gspread_dataframe import set_with_dataframe

# Step 3: Authenticate and create the gspread client
auth.authenticate_user()
creds, _ = default()
creds.refresh(Request())
gc = gspread.authorize(creds)

# Step 4: Open the Google Sheet by URL
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/14eO9Gyo6vjE6hulXd72dyXx8rLFlaI9NACzLlisJAlM/edit?usp=sharing')
worksheet = sheet.get_worksheet(0)  # Open the first sheet

# Step 5: Read data from Google Sheet into pandas DataFrame
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# Print the column names to debug
print("Column names:", df.columns)

# New system prompt
system_prompt = ("You are a subject matter expert creating a test for the purpose of pre-employment screening for a variety of roles and positions. "
                 "Your task is to create highly relevant technical and non-technical questions to evaluate the candidates’ skills in a given subject. "
                 "This is a test and needs to be marked for their overall score, therefore ask questions with a clear correct answer. "
                 "Do not ask questions that ask the candidate to rate themselves, as this will skew the marking criteria. "
                 "Ask questions that have a correct answer or logical questions that can be evaluated. "
                 "Taylor the test to the amount of questions required always provided the required number of questions eg: if requested 30 questions provide 30 question’s and 30 answers. "
                 "Create questions along with their correct answers for marking.")

# Function to convert a row of the DataFrame to a JSON object for OpenAI fine-tuning
def row_to_json(row):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Role: {row['Role']}"},
        {"role": "assistant", "content": f"Question: {row['Question']} Answer: {row['Answer']}"}
    ]
    return {"messages": messages}

# Convert the entire DataFrame to a list of JSON objects
jsonl_data = df.apply(row_to_json, axis=1).tolist()

# Step 6: Format Error Checks Function
def check_format_errors(dataset):
    format_errors = defaultdict(int)
    for ex in dataset:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in ("system", "user", "assistant", "function"):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            function_call = message.get("function_call", None)

            if (not content and not function_call) or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found")

# Step 7: Token Counting Utilities
encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Step 8: Data Warnings and Token Counts Function
def data_warnings_and_token_counts(dataset):
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 16385 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

# Step 9: Cost Estimation Function
def estimate_costs(dataset):
    MAX_TOKENS_PER_EXAMPLE = 16385
    TARGET_EPOCHS = 3
    MIN_TARGET_EXAMPLES = 100
    MAX_TARGET_EXAMPLES = 25000
    MIN_DEFAULT_EPOCHS = 1
    MAX_DEFAULT_EPOCHS = 25

    n_epochs = TARGET_EPOCHS
    n_train_examples = len(dataset)
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

    convo_lens = [num_tokens_from_messages(ex["messages"]) for ex in dataset]
    n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
    print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
    print(f"By default, you'll train for {n_epochs} epochs on this dataset")
    print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Step 10: Execute the functions with the dataset
check_format_errors(jsonl_data)
data_warnings_and_token_counts(jsonl_data)
estimate_costs(jsonl_data)

# Step 11: Save the JSONL data to a file with date and time in the filename
australia_timezone = pytz.timezone('Australia/Sydney')
current_datetime = datetime.now(australia_timezone).strftime('%Y-%m-%d_%H-%M-%S')
output_file = f'fine_tuning_data_{current_datetime}.jsonl'

with open(output_file, 'w') as f:
    for entry in jsonl_data:
        json.dump(entry, f)
        f.write('\n')

print(f'Data has been successfully saved to {output_file}')

# Optional: Download the JSONL file to your local machine
from google.colab import files
files.download(output_file)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.
Column names: Index(['Role', 'Difficulty', 'Question', 'Answer'], dtype='object')
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 201, 433
mean / median: 266.55555555555554, 265.0
p5 / p

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Traning Data - Paul

In [3]:
# Step 1: Install necessary libraries
!pip install --upgrade gspread pandas gspread-dataframe tiktoken pytz --quiet

# Step 2: Import necessary libraries
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
from google.auth import default
import pandas as pd
import json
from collections import defaultdict
import numpy as np
import tiktoken
from datetime import datetime
import pytz
from gspread_dataframe import set_with_dataframe

# Step 3: Authenticate and create the gspread client
auth.authenticate_user()
creds, _ = default()
creds.refresh(Request())
gc = gspread.authorize(creds)

# Step 4: Open the Google Sheet by URL
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1u_ERBCtzbXa1xRUd2LPEfIAHJlNdmvkv7I_D9CgDobA/edit?usp=sharing')  # Replace with your Google Sheet URL
worksheet = sheet.get_worksheet(0)  # Open the first sheet

# Step 5: Read data from Google Sheet into pandas DataFrame
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# Print the column names to debug
print("Column names:", df.columns)

# New system prompt
system_prompt = ("You are a subject matter expert creating a test for the purpose of pre-employment screening for a variety of roles and positions. "
                 "Your task is to create highly relevant technical and non-technical questions to evaluate the candidates’ skills in a given subject. "
                 "This is a test and needs to be marked for their overall score, therefore ask questions with a clear correct answer. "
                 "Do not ask questions that ask the candidate to rate themselves, as this will skew the marking criteria. "
                 "Ask questions that have a correct answer or logical questions that can be evaluated. "
                 "Tailor the test to the amount of questions required, always providing the required number of questions, e.g., if requested 30 questions, provide 30 questions and 30 answers. "
                 "Create questions along with their correct answers for marking.")

# Function to convert a row of the DataFrame to a JSON object for OpenAI fine-tuning
def row_to_json(row):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row['Prompt']},  # User provides the role details
        {"role": "assistant", "content": f"Question: {row['Question']} Answer: {row['Answer']}"}  # Assistant generates question and answer
    ]
    return {"messages": messages}

# Convert the entire DataFrame to a list of JSON objects
jsonl_data = df.apply(row_to_json, axis=1).tolist()

# Step 6: Format Error Checks Function
def check_format_errors(dataset):
    format_errors = defaultdict(int)
    for ex in dataset:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in ("system", "user", "assistant", "function"):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            function_call = message.get("function_call", None)

            if (not content and not function_call) or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found")

# Step 7: Token Counting Utilities
encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Step 8: Data Warnings and Token Counts Function
def data_warnings_and_token_counts(dataset):
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 16385 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 16,385 token limit, they will be truncated during fine-tuning")

# Step 9: Cost Estimation Function
def estimate_costs(dataset):
    MAX_TOKENS_PER_EXAMPLE = 16385
    TARGET_EPOCHS = 3
    MIN_TARGET_EXAMPLES = 100
    MAX_TARGET_EXAMPLES = 25000
    MIN_DEFAULT_EPOCHS = 1
    MAX_DEFAULT_EPOCHS = 25

    n_epochs = TARGET_EPOCHS
    n_train_examples = len(dataset)
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

    convo_lens = [num_tokens_from_messages(ex["messages"]) for ex in dataset]
    n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
    print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
    print(f"By default, you'll train for {n_epochs} epochs on this dataset")
    print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

# Step 10: Execute the functions with the dataset
check_format_errors(jsonl_data)
data_warnings_and_token_counts(jsonl_data)
estimate_costs(jsonl_data)

# Step 11: Save the JSONL data to a file with date and time in the filename
australia_timezone = pytz.timezone('Australia/Sydney')
current_datetime = datetime.now(australia_timezone).strftime('%Y-%m-%d_%H-%M-%S')
output_file = f'fine_tuning_data_{current_datetime}.jsonl'

with open(output_file, 'w') as f:
    for entry in jsonl_data:
        json.dump(entry, f)
        f.write('\n')

print(f'Data has been successfully saved to {output_file}')

# Optional: Download the JSONL file to your local machine
from google.colab import files
files.download(output_file)


Column names: Index(['Prompt', 'Question', 'Answer', 'Difficulty'], dtype='object')
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 207, 293
mean / median: 232.48936170212767, 230.0
p5 / p95: 221.6, 250.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 18, 96
mean / median: 36.04255319148936, 32.0
p5 / p95: 22.6, 53.0

0 examples may be over the 16,385 token limit, they will be truncated during fine-tuning
Dataset has ~10927 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~32781 tokens
Data has been successfully saved to fine_tuning_data_2024-08-28_10-18-29.jsonl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>